In [0]:
dbutils.widgets.text("url_saldo", "")
dbutils.widgets.text("token", "")
dbutils.widgets.text("qtd_meses", "")

In [0]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from typing import List, Dict


def obterPeriodo(qtd_meses: int) -> Dict:
    hoje = date.today()
    data_inicial = (hoje - relativedelta(months=qtd_meses)).replace(day=1)

    return {
        "dataInicial": data_inicial.strftime("%Y-%m-%d"),
        "dataFinal": hoje.strftime("%Y-%m-%d"),
    }


def geraParametrosSaldo(qtd_meses: int = 12) -> List[Dict]:
    """
    Gera uma lista de parâmetros contendo apenas o dia 01 de cada mês.
    """
    periodo = obterPeriodo(qtd_meses)

    data_inicial = datetime.strptime(periodo["dataInicial"], "%Y-%m-%d").date()
    data_final = datetime.strptime(periodo["dataFinal"], "%Y-%m-%d").date()

    parametros = []
    data_atual = data_inicial.replace(day=1)

    while data_atual <= data_final:
        parametros.append(
            {
                "dataInicial": data_atual.strftime("%Y-%m-%d"),
                "dataFinal": data_atual.strftime("%Y-%m-%d"),
            }
        )
        data_atual += relativedelta(months=1)

    return parametros


In [0]:
import requests
from typing import Dict, Optional


def chamar_api(url: str, parametros: Dict, pagina: int = 1) -> Optional[requests.Response]:
    params = parametros.copy()
    params["pagina"] = pagina

    try:
        response = requests.get(url, params=params, timeout=60)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        return None


def obter_saldo_dia_01(url: str, parametros: Dict) -> list:
    """
    Busca saldo para UM dia específico (pagina = 1).
    O controle mensal deve ser feito fora desta função.
    """
    response = chamar_api(url, parametros, pagina=1)

    if response:
        try:
            dados = response.json().get("registros", [])
            return dados
        except (ValueError, KeyError, TypeError):
            print(f"Erro ao interpretar resposta da data {parametros.get('dataInicial')}")

    return []


In [0]:
import json
import time
from datetime import datetime

CATALOG = "bitz"
VOLUME_PATH = f"/Volumes/{CATALOG}/raw/saldo"

URL_SALDO = dbutils.widgets.get("url_saldo")

TOKEN = dbutils.widgets.get("token")
QTD_MESES = int(dbutils.widgets.get("qtd_meses"))

parametros_lista = geraParametrosSaldo(QTD_MESES)

inicio = time.time()
registros = []

for params in parametros_lista:
    params["token"] = TOKEN
    data_ref = params["dataInicial"]

    dados = obter_saldo_dia_01(URL_SALDO, params)

    for item in dados:
        item["_data_referencia"] = data_ref
        item["_ingestion_timestamp"] = datetime.now().isoformat()
        registros.append(item)

fim = time.time()

# PERSISTÊNCIA NO VOLUME (JSON RAW)

if registros:
    data_execucao = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_path = f"{VOLUME_PATH}/saldo_{data_execucao}.json"

    json_content = json.dumps(registros, ensure_ascii=False)

    dbutils.fs.put(
        file_path,
        json_content,
        overwrite=True
    )